
# PRT565 – Assessment 2  
### Machine Learning & Artificial Intelligence  
**Project:** Mental Health in Tech – OSMI Survey (2016–2022)  
**Student:** Surat Adhikari (s376778)  | Master of Data Science


## 1. Problem Description  

Mental health in the tech sector is an important issue. The Open Sourcing Mental Illness (OSMI) survey collects data on how people in tech experience mental health challenges.  

**Goal:** build a machine learning model to predict whether a person has sought treatment (`treatment` column) based on demographics and workplace factors.  

This helps me practice:  
- Cleaning and merging messy, multi-year survey data.  
- Training different models (Logistic Regression, Decision Tree, Random Forest).  
- Evaluating and comparing results using metrics and visuals.  



## 2. Dataset Description  

I used survey data from **2016–2022** (OSMI Mental Health in Tech survey).  
Because questions changed slightly each year, I had to:  
- Standardise column names.  
- Convert Yes/No answers to 1/0.  
- Group gender values into *Male*, *Female*, *Other*.  
- Remove outlier ages (<15 or >100).  

After cleaning, I combined all years into one dataset for modelling.  


# Mental Health in the Technology Industry

About 20% of adults struggle with mental health conditions like ADHD, bipolar disorder, anxiety, or depression. People who are most at risk for mental health issues need to be supported and not left behind in the tech sector, which is marked by a high-stress work environment, long hours, constant availability expectations, and a blending of personal and professional life because of open-source commitments. This can be accomplished by encouraging work-life balance, de-stigmatizing mental health discussions, implementing proactive mental health initiatives, and providing tech workers who are struggling with these issues with support and resources.

# Goal

The purpose of this notebook is to shed light on the mental health situation in the sector between 2014 and 2023 and offer suggestions for a more sustainable and healthy future.

The information provided here comes from Open Source Mental Illness (OSMI) surveys that were carried out from 2014 to 2023. The purpose of these surveys was to assess how people view mental health and the frequency of mental health conditions in the tech sector.

## Problem Description

 to look into the relationships between mental health issues and organisational, social, occupational, and demographic factors.  In light of this, the objective is to create a predictive model that can determine if an individual may require medical assistance.

 ## Justification for Relevance

 Research shows that a person's mental health has a direct effect on how well they perform at work (Paul, M., & Das, S. (2023)).  Furthermore, a database containing hundreds of records gathered from surveys of employees in the technology sector was released by an organisation named Open Sourcing Mental Health (OSMI).  Respondents to this survey provided details about their workplace.  For instance, details regarding the number of reported employees, gender, company benefits, and whether the individual had previously sought medical attention were reported.

 Along with the importance of mental health in the workplace, this database creates the primary driving force behind the advancement of this study.

 ## Suggested Resolution

 The study will be viewed as a classification problem in which it will be determined whether or not a person needs medical assistance based on data gathered from the survey.
 Based on a limited analysis of the data, it was initially determined that the variable to predict would be "treatment" because it indicates whether a person has sought medical attention in the past.  However, it should be noted that this study is merely an approach to the phenomenon under study, with no intention of proving its causality, due to the subjective nature of this self-reported variable (since an individual with mental health issues might not be aware of their illness).


 ## Citations
 Das, S., and Paul, M. (2023).  An analysis of mental health in the tech industry.  10(1), 221-233, International Journal of Science and Research Archive.